In [57]:
import json
import csv
import os
import pandas as pd
from pathlib import Path

In [58]:

folder_path = "v_0.2"
files = os.listdir(folder_path)
print("Files in folder:", files)

csv_path = "v_0.2/paax_v0.2_wide.csv"
xlsx_path = "v_0.2/paax_v0.2.xlsx"

for f in files:
    if f.lower() == "a.csv":
        csv_path = os.path.join(folder_path, f)
    elif f.lower() == "b.xlsx":
        xlsx_path = os.path.join(folder_path, f)

print("CSV file found:", csv_path)
print("Excel file found:", xlsx_path)

if csv_path:
    # 含有常任理事国标识
    df_csv_p5 = pd.read_csv(csv_path)
    display(df_csv_p5.head(2))
else:
    print("country agt file not found.")

if xlsx_path:
    # 网络形式的数据
    df_xlsx_network = pd.read_excel(xlsx_path)
    display(df_xlsx_network.head(2))
else:
    print("network file not found.")

Files in folder: ['pax_id_to_con_info.csv', 'paax_actors_v0.2.csv', 'paax_signatory_v0.2.csv', 'pax_data_2144_agreements_v9.csv', 'paax_signatory_v0.2_internal.csv', 'paax_signatory_v_0.2_intra_interintra_international.csv', 'paax_v0.2_wide.csv', '~$paax_v0.2.xlsx', 'actors_alt_names_211125.csv', 'paax_v0.2.xlsx', 'agts_not_included.csv', 'paax_third_parties_v_0.2_wide.csv', 'paax_actors_v0.2_internal.csv', 'paax_signatory_v_0.2_intrastate_international.csv']
CSV file found: v_0.2/paax_v0.2_wide.csv
Excel file found: v_0.2/paax_v0.2.xlsx


,AgtId,has_third_party,IGO,EU,regional,org_region,UN,UN Office,UN Mission,UNSC,...,TjRep,TjRSym,TjRMa,TjNR,ImUN,ImOth,ImRef,ImPK,ImE,ImSrc
0,2,1,1,1,1,Europe,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,3,1,0,0,0,NaN,0,0,0,0,...,0,0,0,1,1,1,0,1,0,1


,signatory_id,AgtId,agt_name,signatory_type,practical_third,condition,actor_id,id_paax,actor_name,actor_type,...,Loc1ISO,Loc2ISO,Loc1GWNO,Loc2GWNO,UcdpCon,UcdpAgr,PamAgr,CowWar,PAX_Hyperlink,PDF_Hyperlink
0,0,2,Dayton Agreement on Implementing the Federatio...,party,NaN,NaN,0,CON_0,Bosnia and Herzegovina,Country/State,...,BIH,NaN,346.0,NaN,398.0,NaN,NaN,215,https://www.peaceagreements.org/agreements/2/,https://www.peaceagreements.org/media/document...
1,2,2,Dayton Agreement on Implementing the Federatio...,party,1.0,C4,1,CON_1,Croatia,Country/State,...,BIH,NaN,346.0,NaN,398.0,NaN,NaN,215,https://www.peaceagreements.org/agreements/2/,https://www.peaceagreements.org/media/document...


In [59]:
# Fill missing dates in df_xlsx_network using pax_data_2144_agreements_v9.csv
pax_path = Path("v_0.2/pax_data_2144_agreements_v9.csv")
pax_agts = pd.read_csv(pax_path)

if 'Dat' not in df_xlsx_network.columns:
    raise KeyError("df_xlsx_network is missing 'Dat' column")

# Parse network dates: prefer ISO YYYY-MM-DD, then try day-first fallback
df_xlsx_network['Dat'] = pd.to_datetime(df_xlsx_network['Dat'], format='%Y-%m-%d', errors='coerce')
mask_net_nat = df_xlsx_network['Dat'].isna()
if mask_net_nat.any():
    df_xlsx_network.loc[mask_net_nat, 'Dat'] = pd.to_datetime(
        df_xlsx_network.loc[mask_net_nat, 'Dat'], dayfirst=True, errors='coerce'
    )

# Parse pax dates: handle both ISO and day-first slashed inputs
pax_agts['Dat'] = pd.to_datetime(pax_agts['Dat'], format='%Y-%m-%d', errors='coerce')
mask_pax_nat = pax_agts['Dat'].isna()
if mask_pax_nat.any():
    pax_agts.loc[mask_pax_nat, 'Dat'] = pd.to_datetime(
        pax_agts.loc[mask_pax_nat, 'Dat'], dayfirst=True, errors='coerce'
    )

pax_dat_by_agt = pax_agts.dropna(subset=['Dat']).groupby('AgtId')['Dat'].first()

mask_missing = df_xlsx_network['Dat'].isna()
filled_dates = df_xlsx_network.loc[mask_missing, 'AgtId'].map(pax_dat_by_agt)
df_xlsx_network.loc[mask_missing, 'Dat'] = filled_dates

# Standardize output format to dd/mm/yyyy
df_xlsx_network['Dat'] = df_xlsx_network['Dat'].dt.strftime('%d/%m/%Y')

print(f"Filled {filled_dates.notna().sum()} missing dates from pax_data_2144_agreements_v9.csv")


Filled 5405 missing dates from pax_data_2144_agreements_v9.csv


/var/folders/w1/rjy36_sj7gsc6jdr7lrf77d80000gn/T/ipykernel_39044/2836095842.py:3: DtypeWarning: Columns (23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  pax_agts = pd.read_csv(pax_path)


In [60]:
# 用常任理事国过滤国家数据
df_csv_p5 = pd.read_csv("v_0.2/paax_v0.2_wide.csv")
df_un = df_csv_p5[df_csv_p5["UN"] == 1]
df_china = df_csv_p5[df_csv_p5["China"] == 1]
df_russia = df_csv_p5[df_csv_p5["Russia"] == 1]
df_uk = df_csv_p5[df_csv_p5["United Kingdom"] == 1]

count_un_agt = len(df_un)
count_china_agt = len(df_china)
count_russia_agt = len(df_russia)
count_uk_agt = len(df_uk)

print("UN agt count:", count_un_agt,
      "China agt count:", count_china_agt,
      "Russia agt count:", count_russia_agt,
      "UK agt count:", count_uk_agt)


UN agt count: 456 China agt count: 56 Russia agt count: 181 UK agt count: 96


In [61]:
# 用总表格得出国家表格
df_qatar = df_xlsx_network[df_xlsx_network["actor_country"] == "Qatar"]
unique_ids_qatar = df_qatar["AgtId"].nunique()

df_un = df_xlsx_network[df_xlsx_network["un_type"].fillna("").isin(["United Nations", "UN Mission"])]
unique_ids_un = df_un["AgtId"].nunique()
print("UN rows:", len(df_un), "unique AgtId:", unique_ids_un)
print("Qatar rows:", len(df_qatar), "unique AgtId:", unique_ids_qatar)

UN rows: 529 unique AgtId: 456
Qatar rows: 26 unique AgtId: 26


In [62]:
def build_network_subset(country_df, network_df):
    ids = country_df["AgtId"].dropna().unique()
    return network_df[network_df["AgtId"].isin(ids)]

network_un = build_network_subset(df_un, df_xlsx_network)
network_china = build_network_subset(df_china, df_xlsx_network)
network_russia = build_network_subset(df_russia, df_xlsx_network)
network_uk = build_network_subset(df_uk, df_xlsx_network)
network_qatar = build_network_subset(df_qatar, df_xlsx_network)

print("Network UN rows:", len(network_un), "unique AgtId:", network_un["AgtId"].nunique())
print("Network China rows:", len(network_china), "unique AgtId:", network_china["AgtId"].nunique())
print("Network Russia rows:", len(network_russia), "unique AgtId:", network_russia["AgtId"].nunique())
print("Network UK rows:", len(network_uk), "unique AgtId:", network_uk["AgtId"].nunique())
print("Network Qatar rows:", len(network_qatar), "unique AgtId:", network_qatar["AgtId"].nunique())


# If you have others (df_china, df_france, etc.), reuse:
# network_china = build_network_subset(df_china, df_xlsx_network)

Network UN rows: 3918 unique AgtId: 456
Network China rows: 1048 unique AgtId: 56
Network Russia rows: 1575 unique AgtId: 181
Network UK rows: 1415 unique AgtId: 96
Network Qatar rows: 447 unique AgtId: 26


In [63]:
network_sets = {
    "United_Nations": network_un,
    "China": network_china,
    "Russia": network_russia,
    "United_Kingdom": network_uk,
    "Qatar": network_qatar
}

out_dir = Path("intermediary")
out_dir.mkdir(exist_ok=True)

for country, df_net in network_sets.items():
    fname = f"all_agts_with_{country.replace(' ', '_')}.csv"
    df_net.to_csv(out_dir / fname, index=False)
    print(f"saved {fname} with {len(df_net)} rows")

saved all_agts_with_United_Nations.csv with 3918 rows
saved all_agts_with_China.csv with 1048 rows
saved all_agts_with_Russia.csv with 1575 rows
saved all_agts_with_United_Kingdom.csv with 1415 rows
saved all_agts_with_Qatar.csv with 447 rows


In [64]:
actors = pd.read_csv("v_0.2/paax_actors_v0.2.csv")

agt_dir = Path("intermediary")
out_dir = Path("intermediary")
out_dir.mkdir(exist_ok=True)

for path in agt_dir.glob("all_agts_with_*.csv"):
    country = path.stem.replace("all_agts_with_", "")
    network_df = pd.read_csv(path)

    actor_ids = network_df["actor_id"].dropna().unique()
    actors_subset = actors[actors["actor_id"].isin(actor_ids)]

    out_path = out_dir / f"actors_with_{country}.csv"
    actors_subset.to_csv(out_path, index=False)
    print(f"{country}: {len(actors_subset)} rows -> {out_path}")

United_Kingdom: 262 rows -> intermediary/actors_with_United_Kingdom.csv
Qatar: 197 rows -> intermediary/actors_with_Qatar.csv
United_Nations: 641 rows -> intermediary/actors_with_United_Nations.csv
China: 234 rows -> intermediary/actors_with_China.csv
Russia: 226 rows -> intermediary/actors_with_Russia.csv
